In [ ]:
import os
import numpy as np
import mne
from scipy.io import loadmat
from tqdm import tqdm

mne.set_log_level('ERROR')

data_path = './BCICIV_2a_gdf'

X_train_raws = []
Y_train_raws = []
X_test_raws = []
Y_test_raws = []
for file_id in range(1, 10):
    for use_train in [True, False]:
        # Read X
        X_filename = os.path.join(data_path, f'A0{file_id}{["E", "T"][use_train]}.gdf')
        X_raw = mne.io.read_raw_gdf(X_filename)
        X_raw.load_data()

        # Read Y
        Y_filename = os.path.join(data_path, f'A0{file_id}{["E", "T"][use_train]}.mat')
        Y_raw = loadmat(Y_filename)['classlabel'].T.squeeze()

        if use_train:
            X_train_raws.append(X_raw)
            Y_train_raws.append(Y_raw)
        else:
            X_test_raws.append(X_raw)
            Y_test_raws.append(Y_raw)

# Reindex labels from 0 to 3
Y_train_raw = np.hstack(Y_train_raws) - 1
Y_test_raw = np.hstack(Y_test_raws) - 1

In [ ]:
import pywt
from sklearn import preprocessing

def filter_epochs_comb(raws, tmin=1.0, tmax=4.0):
    X = []
    for raw in raws:
        # Filter the raw signal with a band pass filter in 7-35 Hz
        # 8 - 30
        raw.filter(8., 30., fir_design='firwin')

        # Remove the EOG channels and pick only desired EEG channels
        raw.info['bads'] += ['EOG-left', 'EOG-central', 'EOG-right']
        picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False, exclude='bads')

        # Find the events time positions
        events, events_id = mne.events_from_annotations(raw)

        # 1. left_hand = 769, 2. right_hand = 770, 3. foot = 771, 4. tongue = 772
        events_key = ['769', '770', '771', '772'] if use_train else ['768']
        use_events_id = {key: events_id[key] for key in events_key}

        # Extracts epochs of from time peroid of between tmin and tmax from the datset into 288 events for all 4 classes
        epochs = mne.Epochs(raw, events, use_events_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)
        X.append(epochs.get_data())
    X = np.vstack(X)

    return X

def feature_bands(X):
    X_wpds = [[[None for j in range(X.shape[1])] for i in range(X.shape[0])] for b in range(1, 9)]
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # signal is decomposed to level 5 with 'db4' wavelet
            C = pywt.WaveletPacket(X[i, j, :], 'db4', mode='symmetric', maxlevel=5)
            pos = [node.path for node in C.get_level(5, 'natural')]

            for b in range(1, 9):
                data = C[pos[b]].data
                X_wpds[b - 1][i][j] = data
    X_wpds = np.array(X_wpds)

    return X_wpds

def fit_CSP(X, Y, n_comps=4):
    csps = []
    for i in range(8):
        csp = mne.decoding.CSP(n_components=n_comps, reg=None, log=True, norm_trace=False)
        csp.fit(X[i, :, :, :], Y)
        csps.append(csp)

    return csps

def mini_epochs(X_train, X_test, num_epochs=6, interval=None):
    '''
    (num_epochs - 1) * increment + interval = 3s

    original: num_epochs=1
    fixed-window: num_epochs=n
    sliding-window: num_epochs=n, interval=t
    '''

    if interval is None:
        interval = 3 / num_epochs
        increment = interval
    else:
        increment = (3 - interval) / (num_epochs - 1)

    X_train_epochs = []
    X_test_epochs = []
    for tmin in tqdm(np.arange(1, 4 - interval + increment, increment)):
        X_train_filter = filter_epochs_comb(X_train, tmin=tmin, tmax=tmin + interval)
        X_test_filter = filter_epochs_comb(X_test, tmin=tmin, tmax=tmin + interval)

        X_train_wpd = feature_bands(X_train_filter)
        X_test_wpd = feature_bands(X_test_filter)

        csps = fit_CSP(X_train_wpd, Y_train_raw, n_comps=16)
        X_train_csp = np.concatenate([csps[i].transform(X_train_wpd[i, :, :, :]) for i in range(8)], axis=-1)
        X_test_csp = np.concatenate([csps[i].transform(X_test_wpd[i, :, :, :]) for i in range(8)], axis=-1)

        X_train_epochs.append(X_train_csp)
        X_test_epochs.append(X_test_csp)

    num_feats = list(map(lambda x: x.shape[1], X_train_epochs))
    X_train_epochs = np.hstack(X_train_epochs)
    X_test_epochs = np.hstack(X_test_epochs)

    return X_train_epochs, X_test_epochs, num_feats

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.linear_model import LogisticRegression

class MiniEpochEnsemble(BaseEstimator, ClassifierMixin):
    def __init__(self, num_feats, learner='xgb'):
        self.num_feats = num_feats
        self.learner = learner

        ends = np.cumsum(num_feats)
        self.input_ranges = [(0, ends[i]) if i == 0 else (ends[i - 1], ends[i]) for i in range(len(ends))]
        self.base_learners = [self._learner_get() for i in range(len(num_feats))]
        self.ense_learner = None

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y

        margins = []
        for input_range, base_learner in tqdm(zip(self.input_ranges, self.base_learners)):
            X_mini = X[:, input_range[0]:input_range[1]]
            self._learner_fit(X_mini, Y_train, base_learner)
            margins.append(self._learner_margin(X_mini, base_learner))
        margins = np.array(margins)

        self.ense_learner = LogisticRegression(max_iter=1000, random_state=0)
        self.ense_learner.fit(np.hstack(margins), Y_train)

        return self

    def predict(self, X):
        check_is_fitted(self)
        X = check_array(X)

        margins = []
        for input_range, base_learner in tqdm(zip(self.input_ranges, self.base_learners)):
            X_mini = X[:, input_range[0]:input_range[1]]
            margins.append(self._learner_margin(X_mini, base_learner))
        margins = np.array(margins)

        Y = self.ense_learner.predict(np.hstack(margins))

        return Y

    def _learner_get(self):
        if self.learner == 'svm':
            clf = SVC()
        elif self.learner == 'xgb':
            clf = XGBClassifier(learning_rate=3e-1, max_depth=4, reg_lambda=1e2)
        elif self.learner == 'fcn':
            clf = MLPClassifier(alpha=1e-2, hidden_layer_sizes=(50, 50, 50), random_state=1)
        else:
            raise ValueError(f'Base learner {self.learner} not supported!')

        return clf

    def _learner_fit(self, X, Y, clf):
        if self.learner == 'svm':
            clf.fit(X, Y)
        elif self.learner == 'xgb':
            clf.fit(X, Y)
        elif self.learner == 'fcn':
            clf.fit(X, Y)
        else:
            raise ValueError(f'Base learner {self.learner} not supported!')

        return clf

    def _learner_margin(self, X, clf):
        if self.learner == 'svm':
            scores = clf.decision_function(X)
        elif self.learner == 'xgb':
            scores = clf.predict(X, output_margin=True)
        elif self.learner == 'fcn':
            scores = clf.predict_proba(X)
        else:
            raise ValueError(f'Base learner {self.learner} not supported!')

        return scores

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

def split_dataset(X_train_raw, Y_train_raw, X_test_raw, Y_test_raw, test_size=0.2):
    # X = np.vstack((X_train_raw, X_test_raw))
    # Y = np.hstack((Y_train_raw, Y_test_raw))
    # X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

    # X_train = normalize(X_train)
    # X_test = normalize(X_test)

    X_train = normalize(X_train_raw)
    X_test = normalize(X_test_raw)
    Y_train = Y_train_raw
    Y_test = Y_test_raw

    return X_train, Y_train, X_test, Y_test

def eval_clf(X_train, Y_train, X_test, Y_test, clf):
    clf.fit(X_train, Y_train)
    train_accu = clf.score(X_train, Y_train)
    test_accu = clf.score(X_test, Y_test)

    return train_accu, test_accu

## Original dataset

In [ ]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

X_train_orig, X_test_orig, num_feats_fwin = mini_epochs(X_train_raws, X_test_raws, num_epochs=1)
X_train, Y_train, X_test, Y_test = split_dataset(X_train_orig, Y_train_raw, X_test_orig, Y_test_raw)

mlp = MLPClassifier(alpha=1e-2, hidden_layer_sizes=(50, 50, 50), random_state=1)
mlp_scores = eval_clf(X_train, Y_train, X_test, Y_test, mlp)

svm = SVC(C=1)
svm_scores = eval_clf(X_train, Y_train, X_test, Y_test, svm)

xgb = XGBClassifier(learning_rate=3e-1, max_depth=4, reg_lambda=1e2)
xgb_scores = eval_clf(X_train, Y_train, X_test, Y_test, xgb)

orig_scores = [mlp_scores, svm_scores, xgb_scores]

100%|██████████| 1/1 [00:52<00:00, 52.42s/it]
/home/l1qiao/Projects/venv3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
with open('orig_scores.npy', 'wb') as file:
    np.save(file, orig_scores)

## Fixed window

In [ ]:
fwin_scores = []
for num_epochs in [3, 6, 12]:
    X_train_fwin, X_test_fwin, num_feats_fwin = mini_epochs(X_train_raws, X_test_raws, num_epochs=num_epochs)
    X_train, Y_train, X_test, Y_test = split_dataset(X_train_fwin, Y_train_raw, X_test_fwin, Y_test_raw)

    mlp = MLPClassifier(alpha=1e-2, hidden_layer_sizes=(50, 50, 50), random_state=1)
    mlp_scores = eval_clf(X_train, Y_train, X_test, Y_test, mlp)

    svm = SVC(C=1)
    svm_scores = eval_clf(X_train, Y_train, X_test, Y_test, svm)

    xgb = XGBClassifier(learning_rate=3e-1, max_depth=4, reg_lambda=1e2)
    xgb_scores = eval_clf(X_train, Y_train, X_test, Y_test, xgb)

    mlp_ense = MiniEpochEnsemble(num_feats_fwin, 'fcn')
    mlp_ense_scores = eval_clf(X_train, Y_train, X_test, Y_test, mlp_ense)

    svm_ense = MiniEpochEnsemble(num_feats_fwin, 'svm')
    svm_ense_scores = eval_clf(X_train, Y_train, X_test, Y_test, svm_ense)

    xgb_ense = MiniEpochEnsemble(num_feats_fwin, 'xgb')
    xgb_ense_scores = eval_clf(X_train, Y_train, X_test, Y_test, xgb_ense)

    fwin_scores.append([mlp_scores, svm_scores, xgb_scores, mlp_ense_scores, svm_ense_scores, xgb_ense_scores])
fwin_scores

100%|██████████| 3/3 [02:32<00:00, 50.83s/it]
0it [00:00, ?it/s]/home/l1qiao/Projects/venv3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
1it [00:05,  5.66s/it]/home/l1qiao/Projects/venv3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
2it [00:11,  5.61s/it]/home/l1qiao/Projects/venv3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
3it [00:16,  5.58s/it]
3it [00:00, 243.64it/s]
3it [00:00, 253.51it/s]
3it [00:02,  1.25it/s]
3it [00:01,  2.51it/s]
3it [00:01,  2.51it/s]
3it [

[[(1.0, 0.4363425925925926),
  (0.9000771604938271, 0.4957561728395062),
  (1.0, 0.48842592592592593),
  (1.0, 0.3753858024691358),
  (0.910108024691358, 0.42978395061728397),
  (1.0, 0.4471450617283951)],
 [(1.0, 0.44328703703703703),
  (0.9618055555555556, 0.5142746913580247),
  (1.0, 0.4903549382716049),
  (0.9996141975308642, 0.3765432098765432),
  (0.9741512345679012, 0.4166666666666667),
  (1.0, 0.4818672839506173)],
 [(1.0, 0.4756944444444444),
  (0.9884259259259259, 0.5300925925925926),
  (1.0, 0.5061728395061729),
  (0.9984567901234568, 0.4174382716049383),
  (1.0, 0.4077932098765432),
  (1.0, 0.48302469135802467)]]

In [ ]:
with open('fwin_scores.npy', 'wb') as file:
    np.save(file, fwin_scores)

## Sliding window

In [ ]:
swin_scores = []
for interval in [0.75, 1, 1.25]:
    X_train_swin, X_test_swin, num_feats_swin = mini_epochs(X_train_raws, X_test_raws, num_epochs=6, interval=interval)
    X_train, Y_train, X_test, Y_test = split_dataset(X_train_swin, Y_train_raw, X_test_swin, Y_test_raw)

    mlp = MLPClassifier(alpha=1e-2, hidden_layer_sizes=(50, 50, 50), random_state=1)
    mlp_scores = eval_clf(X_train, Y_train, X_test, Y_test, mlp)

    svm = SVC(C=1)
    svm_scores = eval_clf(X_train, Y_train, X_test, Y_test, svm)

    xgb = XGBClassifier(learning_rate=3e-1, max_depth=4, reg_lambda=1e2)
    xgb_scores = eval_clf(X_train, Y_train, X_test, Y_test, xgb)

    mlp_ense = MiniEpochEnsemble(num_feats_swin, 'fcn')
    mlp_ense_scores = eval_clf(X_train, Y_train, X_test, Y_test, mlp_ense)

    svm_ense = MiniEpochEnsemble(num_feats_swin, 'svm')
    svm_ense_scores = eval_clf(X_train, Y_train, X_test, Y_test, svm_ense)

    xgb_ense = MiniEpochEnsemble(num_feats_swin, 'xgb')
    xgb_ense_scores = eval_clf(X_train, Y_train, X_test, Y_test, xgb_ense)

    swin_scores.append([mlp_scores, svm_scores, xgb_scores, mlp_ense_scores, svm_ense_scores, xgb_ense_scores])
swin_scores

100%|██████████| 6/6 [05:01<00:00, 50.28s/it]
0it [00:00, ?it/s]/home/l1qiao/Projects/venv3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
1it [00:05,  5.83s/it]/home/l1qiao/Projects/venv3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
2it [00:11,  5.59s/it]/home/l1qiao/Projects/venv3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
3it [00:16,  5.55s/it]/home/l1qiao/Projects/venv3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: Convergen

[[(1.0, 0.44367283950617287),
  (0.9444444444444444, 0.5169753086419753),
  (1.0, 0.5227623456790124),
  (0.9949845679012346, 0.4378858024691358),
  (0.9598765432098766, 0.44328703703703703),
  (1.0, 0.5084876543209876)],
 [(1.0, 0.4737654320987654),
  (0.9293981481481481, 0.5254629629629629),
  (1.0, 0.5162037037037037),
  (0.996141975308642, 0.42592592592592593),
  (0.9444444444444444, 0.435570987654321),
  (1.0, 0.498070987654321)],
 [(1.0, 0.4984567901234568),
  (0.9286265432098766, 0.560570987654321),
  (1.0, 0.5582561728395061),
  (0.9845679012345679, 0.5),
  (0.9436728395061729, 0.4699074074074074),
  (1.0, 0.5536265432098766)]]

In [ ]:
with open('swin_scores.npy', 'wb') as file:
    np.save(file, swin_scores)

## Table 1

In [ ]:
import pandas as pd

table1 = []

orig_row = []
for orig_score in orig_scores:
    orig_row.append(orig_score[1])
table1.append(orig_row)

for fwin_score in fwin_scores:
    fwin_row = []
    for i in range(3):
        fwin_row.append(fwin_score[i][1])
    table1.append(fwin_row)

for swin_score in swin_scores:
    swin_row = []
    for i in range(3):
        swin_row.append(swin_score[i][1])
    table1.append(swin_row)

columns = ['MLP', 'SVM', 'XGB']
index=['Original', 'Fixed N=3', 'Fixed N=6', 'Fixed N=12', 'Sliding T=0.75s', 'Sliding T=1.00s', 'Sliding T=1.25s']
pd.DataFrame(np.array(table1) * 100, columns=columns, index=index).applymap("{0:.2f}%".format)

,MLP,SVM,XGB
Original,37.35%,40.47%,38.12%
Fixed N=3,43.63%,49.58%,48.84%
Fixed N=6,44.33%,51.43%,49.04%
Fixed N=12,47.57%,53.01%,50.62%
Sliding T=0.75s,44.37%,51.70%,52.28%
Sliding T=1.00s,47.38%,52.55%,51.62%
Sliding T=1.25s,49.85%,56.06%,55.83%


## Table 2

In [ ]:
table2 = []

for fwin_score in fwin_scores:
    fwin_row = []
    for i in range(3, 6):
        fwin_row.append(fwin_score[i][1])
    table2.append(fwin_row)

for swin_score in swin_scores:
    swin_row = []
    for i in range(3, 6):
        swin_row.append(swin_score[i][1])
    table2.append(swin_row)

columns = ['MLP', 'SVM', 'XGB']
index=['Fixed N=3', 'Fixed N=6', 'Fixed N=12', 'Sliding T=0.75s', 'Sliding T=1.00s', 'Sliding T=1.25s']
pd.DataFrame(np.array(table2) * 100, columns=columns, index=index).applymap("{0:.2f}%".format)

,MLP,SVM,XGB
Fixed N=3,37.54%,42.98%,44.71%
Fixed N=6,37.65%,41.67%,48.19%
Fixed N=12,41.74%,40.78%,48.30%
Sliding T=0.75s,43.79%,44.33%,50.85%
Sliding T=1.00s,42.59%,43.56%,49.81%
Sliding T=1.25s,50.00%,46.99%,55.36%
